<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/text_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Key Words & Documents Clusters

In [1]:
import pandas as pd

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving beer_5k.csv to beer_5k.csv


In [3]:
# Dataframe processing
df = pd.read_csv('beer_5k.csv')
texts = [item.replace("\t", " ")[:3000] for item in df['text'] [:3000]]
#df.head(2)

In [4]:
#Create a single blob of text
corpus = ' '.join(texts)

In [5]:
%%capture
!pip install yake

In [6]:
import yake

In [7]:
# YAKE Config
kw_extractor = yake.KeywordExtractor()
language = 'en'
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 50

In [8]:
#Keyword for the corpus
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(corpus)
keywords

[('light beer', 1.285862110631933e-05),
 ('beer', 1.612032579725628e-05),
 ('light', 1.8648034783039633e-05),
 ('IPA', 2.6410999798696757e-05),
 ('white head', 2.773637528095163e-05),
 ('hops', 3.169211523183835e-05),
 ('Pale Ale', 3.2382000613007654e-05),
 ('head', 3.309648906908523e-05),
 ('Caldera IPA', 3.82692808917421e-05),
 ('taste', 4.016587806081062e-05),
 ('beer pours', 4.332471290179747e-05),
 ('malt', 4.340184406837427e-05),
 ('nice', 4.410993501193543e-05),
 ('good beer', 4.652402293074225e-05),
 ('Amstel Light', 4.7792729250408825e-05),
 ('hop', 4.843647543369399e-05),
 ('good', 4.88820811592935e-05),
 ('sweet malt', 5.397537099717455e-05),
 ('Good IPA', 5.554913619317332e-05),
 ('hop flavor', 5.850665964096011e-05),
 ('pours', 6.413223278540112e-05),
 ('hop aroma', 7.863303490032584e-05),
 ('flavor', 8.67756300301069e-05),
 ('AMerican IPA', 9.011714646720996e-05),
 ('light malt', 9.085433519406331e-05),
 ('nice IPA', 9.368885285353805e-05),
 ('aroma', 0.000101088516399438

In [9]:
#Keyword for the individual documents
custom_kw_extractor_docs = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=10, features=None)

In [12]:
#Keywords per document
distilled_docs = []
for item in texts:
  keywords = custom_kw_extractor_docs.extract_keywords(item)
  tmp = [item[0] for item in keywords]
  distilled_docs.append(tmp)

In [16]:
ipa_class = []
for ind, item in enumerate(distilled_docs):
  if 'IPA' in item:
    ipa_class.append((ind,item))

In [18]:
ipa_class[10]

(346,
 ['Reviewed',
  'Decent IPA',
  'hops',
  'citrus',
  'bitter',
  'IPA',
  'off-white head',
  'Pours',
  'head',
  'Aroma'])

Transformers

In [ ]:
%pip install -U sentence-transformers

In [ ]:
from transformers import pipeline
%%capture
model = SentenceTransformer('all-MiniLM-L6-v2')

# Useful Code Snippets

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
dataset = [
    "I enjoy reading about Machine Learning and Machine Learning is my PhD subject",
    "I would enjoy a walk in the park",
    "I was reading in the library"
]

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(dataset)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(25))

spaCY

In [ ]:
import spacy 
from spacy.language import Language 

In [ ]:
# text processing      
nlp = spacy.load('en_core_web_sm')        
#df['Tokens'] = df.Sumcription.apply(lambda x: nlp(x))    
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS        
#spacy_stopwords.add('attach')
#df['Lema_Token']  = df.Tokens.apply(lambda x: " ".join([token.lemma_ for token in x if token not in spacy_stopwords]))

In [ ]:
@Language.component("info_component")
def my_component(doc):
    #print(f"After tokenization, this doc has {len(doc)} tokens.")
    print("The lemmatized tokens are:", [token.lemma_ for token in doc])
    return doc

nlp = spacy.load("en_core_web_sm", disable = ['ner'])
pipeline = ["tok2vec", "parser", "lemmatizer"]

nlp.add_pipe("info_component", name="print_info",last=True)
print(nlp.pipe_names)  # ['tagger', 'parser', 'ner', 'print_info']
doc = nlp("This is a sentence.")

In [ ]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
tokens = [token.lemma_ for token in doc if token.lemma_ not in spacy_stopwords and not token.is_punct]  

In [ ]:
#tensors
vectors = [item.tensor for item in doc]
# not lemmatized tokens
tokens = [item.text for item in doc]

In [ ]:
# NLP Pipelien
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a sentence.")
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
for doc in nlp.pipe(texts[:20]):
  print(doc.ents)

In [ ]:
texts_spacy = []
for doc in nlp.pipe(texts[:3000]):
  tokens = [token.lemma_ for token in doc if token.lemma_ not in spacy_stopwords and not token.is_punct]

  texts_spacy.append(' '.join(tokens))

In [ ]:
texts_spacy_texts = []
for doc in nlp.pipe(texts[:3000]):
  tokens = [token.text for token in doc if token.text not in spacy_stopwords and not token.is_punct]
  texts_spacy_texts.append(' '.join(tokens))

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(texts[:3000])
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(50))

Rake-NLTK

In [ ]:
%%capture
!pip install rake_nltk

In [ ]:
from rake_nltk import Rake
import nltk
%%capture
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
rake_nltk_var = Rake(include_repeated_phrases=False, min_length=1, max_length=3)
rake_nltk_var.extract_keywords_from_text(long_text)
keyword_extracted = rake_nltk_var.get_ranked_phrases_with_scores()
#print(keyword_extracted)